In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.nonparametric import smoothers_lowess
from pandas import Series, DataFrame
from patsy import dmatrices
from sklearn import datasets, svm
# import xgboost as xgb


/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [48]:
inputfile = "rawData/train.csv"
traindata = pd.read_csv(inputfile)


In [49]:
inputfile2 = "rawData/test.csv"
testdata = pd.read_csv(inputfile2)


In [79]:
traindata.tail(50)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
841,842,0,2,"Mudd, Mr. Thomas Charles",male,16.0,0,0,S.O./P.P. 3,10.5000,NaN,S
842,843,1,1,"Serepeca, Miss. Augusta",female,30.0,0,0,113798,31.0000,NaN,C
843,844,0,3,"Lemberopolous, Mr. Peter L",male,34.5,0,0,2683,6.4375,NaN,C
844,845,0,3,"Culumovic, Mr. Jeso",male,17.0,0,0,315090,8.6625,NaN,S
845,846,0,3,"Abbing, Mr. Anthony",male,42.0,0,0,C.A. 5547,7.5500,NaN,S
846,847,0,3,"Sage, Mr. Douglas Bullen",male,NaN,8,2,CA. 2343,69.5500,NaN,S
847,848,0,3,"Markoff, Mr. Marin",male,35.0,0,0,349213,7.8958,NaN,C
848,849,0,2,"Harper, Rev. John",male,28.0,0,1,248727,33.0000,NaN,S
849,850,1,1,"Goldenberg, Mrs. Samuel L (Edwiga Grabowska)",female,NaN,1,0,17453,89.1042,C92,C
850,851,0,3,"Andersson, Master. Sigvard Harald Elias",male,4.0,4,2,347082,31.2750,NaN,S


In [76]:
traindata.loc[1,'Cabin']

'C85'

In [5]:
df_col = traindata.columns

In [77]:
traindata.loc[traindata['Cabin']!='C85'].index.size

890

In [73]:
traindata.loc[traindata['Cabin']==np.NaN]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [54]:
traindata['PassengerId'].max()

891

In [46]:
traindata.loc[traindata['PassengerId']<=891.0].index.size

891

In [41]:
traindata['PassengerId'].describe()

count    891.000000
mean     446.000000
std      257.353842
min        1.000000
25%      223.500000
50%      446.000000
75%      668.500000
max      891.000000
Name: PassengerId, dtype: float64

In [30]:
traindata.loc[traindata.PassengerId==901]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [26]:
traindata['Cabin'].index.size

891

In [51]:
traindata['Cabin'].describe()

count             204
unique            147
top       C23 C25 C27
freq                4
Name: Cabin, dtype: object

In [6]:
df_col

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [10]:
train_x = traindata.loc[:,['Fare']]

In [8]:
train_y = traindata.loc[:,['Survived']]

In [9]:
train_x.head(5)

,Pclass
0,3
1,1
2,3
3,1
4,3


In [11]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_x, train_y)

In [14]:
test_x = testdata.loc[:,['Fare']]

In [16]:
test_x = test_x.replace(np.NAN,0)

In [17]:
test_y = clf.predict(test_x)

In [20]:
test_y[:5]

array([0, 0, 0, 0, 1])

In [22]:
# pd.DataFrame(test_y).to_csv("rawData/submission.csv",index=False)

In [26]:
outputdf = pd.DataFrame(test_y,columns=['Survived'])

In [27]:
outputdf.head(5)

,Survived
0,0
1,0
2,0
3,0
4,1


In [30]:
outputdf2 = testdata.loc[:,['PassengerId']].merge(outputdf, left_index=True, right_index=True)

In [31]:
outputdf2.head(5)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [33]:
outputdf2.to_csv("submission.csv",index=False)

In [34]:
tt = outputdf2.merge(outputdf2, left_index=True, right_index=True)

In [35]:
tt.head()

,PassengerId_x,Survived_x,PassengerId_y,Survived_y
0,892,0,892,0
1,893,0,893,0
2,894,0,894,0
3,895,0,895,0
4,896,1,896,1
